In [ ]:
%load_ext autoreload
%autoreload 2
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random
random.seed(42)

In [ ]:
datagenerator = DataGenerator()
utils = Utils()

In [ ]:
# dataset and model list / dict
dataset_list = [
    "01_ALOI",
    "02_annthyroid",
    "03_backdoor",
    "04_breastw",
    "05_campaign",
    "06_cardio",
    "07_Cardiotocography",
    "08_celeba",
    "09_census",
    "99_linear",
    "99_circles",
    "99_moons",
    "99_clusters",
    "99_oilflow",
]
dataset = dataset_list[1]
datagenerator.dataset = dataset

data = datagenerator.generator(
    la=0.999,
    realistic_synthetic_mode=None,
    stdscale=True,
    minmax=False,
    noise_type=None,
)

Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
lb_test = torch.tensor(data["y_test"], dtype=torch.float32)
idx_n = np.where(lb_train == 0)[0]
Y_train = Y_train[idx_n]
lb_train = lb_train[idx_n]

In [ ]:
n_train = len(Y_train)
data_dim = Y_train.shape[1]
latent_dim = 3
n_inducing = 50
n_epochs = 1000
nn_layers = 15, 15
lr = 0.01
batch_size = 128

In [ ]:
gplvm = AD_GPLVM(latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size)

In [ ]:
gplvm.fit(Y_train)

In [20]:
std = gplvm.encoder.sigma(Y_train)
my_list = []

In [27]:
for x in std.detach():
    val = torch.sqrt(x.diag())
    my_list.append()

In [30]:
my_list[0]

array([0.50850314, 0.439139  , 0.3943587 ], dtype=float32)

In [ ]:
# score = gplvm.predict_score(Y_test)
# score

In [19]:
# utils.metric(y_true=lb_test, y_score=score)